In [1]:
storage_account = "ds562team9datalake"
container_name = "bronze"
file_path = "Historical_crypto/bitcoin/"  # NOTE: this is a folder of Parquet parts

storage_key = "KXg2Djg7uRevBSpPNIVnKw/N6HpqBh+kJwDX07wkywbpU2joMZdTIBOXk30EoMMxH2d8wwb+9j0g+AStO60IWw=="

spark.conf.set(f"fs.azure.account.key.{storage_account}.blob.core.windows.net", storage_key)

url = f"wasbs://{container_name}@{storage_account}.blob.core.windows.net/{file_path}"

df_bitcoin = spark.read.parquet(url)

df_bitcoin.printSchema()
print("Row count:", df_bitcoin.count())

In [3]:
# Count nulls in each column (skip isnan for non-numeric types)
null_counts = df_bitcoin.select([
    count(when(col(c).isNull(), c)).alias(str(c)) for c in df_bitcoin.columns
])
null_counts.show(truncate=False)

# Get min and max date
df_bitcoin.select(
    min(col("`('Date', '')`")).alias("min_date"),
    max(col("`('Date', '')`")).alias("max_date")
).show()

In [4]:
# Rename columns
df_bitcoin_renamed = df_bitcoin.toDF("Date", "Close", "High", "Low", "Open", "Volume")

# Coalesce to a single file and save to bronze
df_bitcoin_renamed.coalesce(1).write.mode("overwrite").parquet(
    "wasbs://bronze@ds562team9datalake.blob.core.windows.net/Historical_crypto/cleaned_bitcoin_data/"
)